In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.circuit.library import QFT
from math import gcd
from fractions import Fraction
from random import randint

def apply_qft(circuit, n_qubits):
    qft_circuit = QFT(n_qubits)
    circuit.append(qft_circuit, range(n_qubits))
    

def continued_fraction(measurement, n_qubits):
    frac = Fraction(measurement / (2 ** n_qubits)).limit_denominator()
    return frac.denominator


def modular_exponentiation(circuit, a, N, n_qubits):
    for x in range(n_qubits):
        exp = pow(a, 2**x, N)
        for _ in range(exp):
            circuit.x(x)  
    return circuit


def measure(circuit, n_qubits):

    circuit.measure(range(n_qubits), range(n_qubits))

def order_finding(a, N, n_qubits=4):
    if gcd(a, N) != 1:
        return f"{a} and {N} are not coprime."


    circuit = QuantumCircuit(n_qubits, n_qubits)
    circuit = modular_exponentiation(circuit, a, N, n_qubits)
    apply_qft(circuit, n_qubits)
    measure(circuit, n_qubits)

    simulator = AerSimulator()
    job = transpile(circuit, backend=simulator)
    result = simulator.run(job).result()

    
    counts = result.get_counts(circuit)
    # print(counts)
    measurement = max(counts, key=counts.get)
    decimal_measurement = int(measurement, 2)

    
    r = continued_fraction(decimal_measurement, n_qubits)
    return r, counts

def shor_algorithm(N, max_retries=20):
    factors = []

    
    if N % 2 == 0:
        factors.append(2)
        N //= 2

    while N > 1:
        retries = 0


        while retries < max_retries:
            a = randint(2, N - 1)
            if gcd(a, N) != 1:
                continue 

            r, _ = order_finding(a, N)
            if r % 2 == 0 and pow(a, r // 2, N) != N - 1:
                factor1 = gcd(pow(a, r // 2) - 1, N)
                factor2 = gcd(pow(a, r // 2) + 1, N)

            
                if factor1 > 1 and factor1 < N:
                    factors.append(factor1)
                    N //= factor1
                    break
                elif factor2 > 1 and factor2 < N:
                    factors.append(factor2)
                    N //= factor2
                    break

            retries += 1  

        if retries == max_retries:
            factors.append(N)
            break 

    return factors


N = 8765
factors = shor_algorithm(N)
print(f"Factors of {N}: {factors}")


Factors of 8765: [5, 1753]


### **Shor's Algorithm**
- **Purpose**: This function combines both quantum and classical parts to find the factors of a number $N$.
- **Explanation**:
  - The algorithm begins by checking if $N$ is even. If so, it immediately adds $2$ as a factor.
  - Next, it tries random values of $a$ and checks if they are coprime with $N$. If they are, the `order_finding` function is invoked to find the period $r$.
  - If the period $r$ is even and satisfies certain conditions, the algorithm uses the greatest common divisor (GCD) to compute the factors of $N$.
  - This process is repeated until all factors are found or the maximum retry limit is reached.

---

### **Quantum Fourier Transform (QFT)**
- **Purpose**: The QFT is applied to a quantum circuit to help identify the period of a modular exponentiation function.
- **Explanation**: 
  - In Shor's algorithm, the QFT is used to transform a quantum state in such a way that the period of a function becomes visible in the measurement results.
  - This operation is essential for extracting information about the period, which is later used to find the factors of the number being factored.

---

### **Continued Fraction Approximation**
- **Purpose**: This function approximates the measurement result as a fraction and returns the denominator, which is related to the period found in the quantum part of the algorithm.
- **Explanation**:
  - After performing quantum operations, the measurement result is a binary value. This function converts that value into a fraction.
  - The denominator of this fraction is related to the period of the modular exponentiation function, which is central to Shor's algorithm.

---

### **Modular Exponentiation**
- **Purpose**: This function implements modular exponentiation, which calculates powers of a number modulo $N$.
- **Explanation**: 
  - Modular exponentiation is crucial for Shor's algorithm because it generates quantum states that encode the powers of $a$ modulo $N$.
  - This operation helps create a superposition of states, which is used in the subsequent Quantum Fourier Transform (QFT) to reveal the period of the modular function.

---

### **Measurement**
- **Purpose**: This function measures the quantum state in all qubits and stores the result.
- **Explanation**:
  - Once the quantum circuit has been executed (including the modular exponentiation and QFT), the quantum state is measured.
  - The measurement collapses the quantum state into one of its possible outcomes, and the result is used to determine the period.

---

### **Order Finding**
- **Purpose**: This is the main quantum part of Shor's algorithm, where the period $r$ of a modular exponentiation function is found.
- **Explanation**:
  - This function first checks if the number $a$ and $N$ are coprime (i.e., they have no common factors). If they are not, the algorithm can easily find a factor of $N$.
  - If $a$ and $N$ are coprime, the function builds a quantum circuit that performs modular exponentiation and applies the QFT.
  - After executing the quantum circuit and measuring the result, it uses continued fraction approximation to extract the period $r$, which is used to find factors of $N$.

---

### **Overall Flow of the Algorithm**:
1. **Initial Check**: If $N$ is even, the factor $2$ is added directly.
2. **Quantum Circuit**: A quantum circuit is created to perform modular exponentiation and apply the QFT.
3. **Measurement**: The quantum state is measured to reveal a value related to the period.
4. **Classical Post-Processing**: After obtaining the period, classical techniques like GCD are used to extract the factors of $N$.
5. **Factorization**: The algorithm continues to factor $N$ by trying random values of $a$ until all factors are found.
